# **Initialization**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/DSML_LAB_PROJECT/Project_Files'

/content/drive/MyDrive/DSML_LAB_PROJECT/Project_Files


In [3]:
!pip uninstall -y torch torchvision 
!pip install torch==1.1.0   torchvision==0.3.0
!pip install torch-cluster==1.4.2
!pip install torch-geometric==1.3.0
!pip install torch-scatter==1.3.1
!pip install torch-sparse==0.4.0 
!pip install scikit-learn==0.20.3
!pip install pytorch-memlab==0.0.3
%load_ext google.colab.data_table

Uninstalling torch-1.7.0+cu101:
  Successfully uninstalled torch-1.7.0+cu101
Uninstalling torchvision-0.8.1+cu101:
  Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 676.9MB 22kB/s 
     |████████████████████████████████| 2.6MB 6.9MB/s 
     |████████████████████████████████| 716kB 6.5MB/s 
  Created wheel for torch-cluster: filename=torch_cluster-1.4.2-cp36-cp36m-linux_x86_64.whl size=14148214 sha256=ca3cdcfc42780dad1b0c5aa0adfcdf4c527d8674a51736c0c8f8534bed4a320f
  Stored in directory: /root/.cache/pip/wheels/ee/54/4a/eb079ec61377a93ea07aba9eea53adcae41450111b54de0118
Successfully built torch-cluster
     |████████████████████████████████| 112kB 6.6MB/s 
     |████████████████████████████████| 235kB 9.1MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.3.0-cp36-none-any.whl size=176111 sha256=43bb7a2b26576a652de9acdcc70e88873e2a44a6f3037243c644c507b836f8da
  Stored in dire

In [4]:
from data.utils import load_data_torch, process_prot_edge
from torch_geometric.data import Data
from src.utils import process_edges
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import csv
import torch
import os

torch.manual_seed(1111)
np.random.seed(1111)

!!! Note: make sure the scipy and torch packages have also been installed properly to run this notebook.

# **Load index, data and results**

### Id-index map loading

In [5]:
# drug id - index
with open('data/index_map/drug-map.pkl', 'rb') as f:
    drug_map = pickle.load(f)
inv_drug_map = {v: k for k, v in drug_map.items()}

# combo id - index
with open('data/index_map/combo_map.pkl', 'rb') as f:
    combo_map = pickle.load(f)
inv_combo_map = {v: k for k, v in combo_map.items()}

### Generate / use polypharmacy side effect id-name map

In [6]:
######################################################
# generate polypharmacy side effect id - name map
# combo_name_map = {}
# with open('../data/index_map/bio-decagon-combo.csv', 'r') as f:
#     reader = csv.reader(f)
#     next(reader)
#     for _, _, id, name in reader:
#         id = int(id.split('C')[-1])
#         combo_name_map[id] = name

# # save map
# with open('../data/index_map/combo-name-map.pkl', 'wb') as f:
#     pickle.dump(combo_name_map, f)

# use map
with open('data/index_map/combo-name-map.pkl', 'rb') as f:
    combo_name_map = pickle.load(f)
inv_combo_name_map = {v: k for k, v in combo_name_map.items()}

### Data loading with selected d-d edge labels

In [7]:
# selected-drug idx - drug idx
with open('data/decagon_et.pkl', 'rb') as f:   # the whole dataset
    et_list = pickle.load(f)
inv_et_list = {et_list[i]: i for i in range(len(et_list))}

# load training data
feed_dict = load_data_torch("data/", et_list, mono=True)
data = Data.from_dict(feed_dict)

loading data
remove  0  isolated drugs:  []
remove finished
1097  polypharmacy side effects
data has been loaded


In [8]:
import itertools
from src.utils import * 

number_of_drugs = 645
number_of_side_effects = 137
number_of_drug_combos = number_of_drugs*(number_of_drugs-1)/2

iteration = 8
initial = (iteration - 1) * number_of_side_effects
final = (iteration) * number_of_side_effects

if iteration == 8:
  final = final + 1
  number_of_side_effects = number_of_side_effects + 1

data.test_et = torch.zeros([int(number_of_side_effects * number_of_drug_combos)], dtype=torch.int64)
data.test_idx = torch.zeros([2, int(number_of_side_effects * number_of_drug_combos)], dtype=torch.int64)

data.test_idx[0], data.test_idx[1], data.test_et = compute_side_effect_drug_pair_combinations(number_of_drugs, number_of_side_effects, number_of_drug_combos, initial, final)

print(data.test_et.shape)

torch.Size([28661220])


# **Performance Comparison**

In [9]:
# model loading
models = {  'DistMult' : 'df_distmult',
            'R-GCN': 'ddm-df_rgcn', 
            'dTIP-D': 'ddm-nn', 
            'dTIP-p': 'ppm-ggm-nn', 
            'TIP-cat': 'tip-cat',
            'TIP-sum': 'tip-add'    }

# loading function for recorded test scores druging training
def get_test_out(model_name):
    with open('output/'+ models[model_name] +'/test_out.pkl', 'rb') as f:
        record = pickle.load(f)
    return record

In [10]:
# get averaged auprc scores for each epoches
scores = dict()
prc_final = dict()
for model in models.keys():
    out = get_test_out(model)
    scores[model] = out
    prc_final[model] = out[99][0]

In [11]:
# print final AUPRC scores  for each models
lines = '---------------------------------------------'
print(lines)
print('|{:10s}|'.format('AUPRC scores for all the model variants'))
print(lines)
formats = '|{:14s}|{:28.3f}|'
for model, scores in prc_final.items():
    print(formats.format(model, scores))
print(lines)

---------------------------------------------
|AUPRC scores for all the model variants|
---------------------------------------------
|DistMult      |                       0.844|
|R-GCN         |                       0.888|
|dTIP-D        |                       0.885|
|dTIP-p        |                       0.778|
|TIP-cat       |                       0.889|
|TIP-sum       |                       0.889|
---------------------------------------------


# **Model Characteristics and Relablility** 

### The side effects evaluated in Zitnik et al. (2018)
The side effects with the top 10 best performance and 10 best performance, according to averaged auprc scores

In [12]:
# ######################################################
# side effect name - original index reported in decagon
decagon_best_name = ["Mumps", "Carbuncle", "Coccydynia", "Tympanic membrane perfor", "Dyshidrosis", "Spondylosis", "Schizoaffective disorder", "Breast dysplasia", "Ganglion", "Uterine polyp"]
decagon_worst_name = ["Bleeding", "Body temperature increased",  "Emesis", "Renal disorder", "Leucopenia", "Diarrhea", "Icterus", "Nausea", "Itch", "Anaemia"]
decagon_best_org_id = [26780, 7078, 9193, 206504, 32633, 38019, 36337, 16034, 1258666, 156369]
decagon_worst_org_id = [19080, 15967, 42963, 22658, 23530, 11991, 22346, 27497, 33774, 2871]

# get index
decagon_best_idx = [inv_et_list[combo_map[i]] for i in decagon_best_org_id]
decagon_worst_idx = [inv_et_list[combo_map[i]] for i in decagon_worst_org_id]

### The side effects evaluated in our work
The side effects with the top 20 best and 20 worst performance, according to averaged auprc scores.

In [14]:
# ######################################################
# Evaluation
name = 'TIP-sum (PPM-GGM-DDM-DF-Sum)'
lines = '-------------------------------------------------------------------------------------------------------'

with open('output/tip-add/test_record.pkl', 'rb') as f:
    dist_record = pickle.load(f)
auprc = np.array(dist_record[len(dist_record)-1])[0, :]
sorted_idx = np.argsort(auprc, kind='quicksort')

print(lines)
print(' {:37s}   {:6s}| {:45s}  {:6s}'.format('The Highest AUPRC Score', '  Edge', 'The Lowest AUPRC Score', '   Edge'))
print(lines)

for i in range(20):
    print(' {:30s} {:7.4f}  {:6d}| {:38s} {:7.4f}  {:6d}'.format(
        combo_name_map[inv_combo_map[et_list[sorted_idx[-(i+1)]]]], auprc[sorted_idx[-(i+1)]], feed_dict['dd_adj_list'][-(i+1)].nnz,
        combo_name_map[inv_combo_map[et_list[sorted_idx[i]]]], auprc[sorted_idx[i]], feed_dict['dd_adj_list'][i].nnz))
print(lines)

decag_best_in_us = [962 - np.where(sorted_idx == i)[0] for i in decagon_best_idx]
decag_worst_in_us = [np.where(sorted_idx == i)[0] for i in decagon_worst_idx]

-------------------------------------------------------------------------------------------------------
 The Highest AUPRC Score                   Edge| The Lowest AUPRC Score                            Edge
-------------------------------------------------------------------------------------------------------
 Corneal dystrophy               0.9843     293| enterocolitis                           0.7578     387
 heat rash                       0.9812     361| neonatal respiratory distress syndrome  0.7655   12062
 prostatism                      0.9805     339| dyspareunia                             0.7747     356
 periostitis                     0.9795     303| Feeling unwell                          0.7806   19930
 cystic acne                     0.9780     259| hypertrichosis                          0.7810   12309
 Legionella                      0.9766     355| thrombocytopenia                        0.7872   14192
 arterial insufficiency          0.9753     327| acute kidney fa

# **Compiling predicitions across the models - Tabulating the results**

## Loading predictions from pickle, and creating a Dataframe from it 

In [ ]:
import numpy as np

head = ['df_distmult', 'ddm-df_rgcn','ddm-nn','ppm-ggm-nn','tip-cat', 'tip-add']

start = initial * int(number_of_drug_combos)
end = final * int(number_of_drug_combos)

In [ ]:
with open('output/evaluation/df_distmult.pkl', 'rb') as f:
  score1 = np.array(pickle.load(f)[start:end])

In [ ]:
with open('output/evaluation/ddm-df_rgcn.pkl', 'rb') as f:
  score2 = np.array(pickle.load(f)[start:end])

In [ ]:
with open('output/evaluation/ddm-nn.pkl', 'rb') as f:
  score3 = np.array(pickle.load(f)[start:end])

In [ ]:
with open('output/evaluation/ppm-ggm-nn.pkl', 'rb') as f:
  score4 = np.array(pickle.load(f)[start:end])

In [ ]:
with open('output/evaluation/tip-cat.pkl', 'rb') as f:
  score5 = np.array(pickle.load(f)[start:end])

In [ ]:
with open('output/evaluation/tip-add.pkl', 'rb') as f:
  score6 = np.array(pickle.load(f)[start:end])

In [ ]:
# side effect info - index and name
side_effect_idx = [inv_combo_map[i] for i in data.test_et.tolist()]
side_effect_name = [combo_name_map[i] for i in side_effect_idx]

In [ ]:
# drug info - PubChem index
drug1_cid = [inv_drug_map[i] for i in data.test_idx[0].tolist()]
drug2_cid = [inv_drug_map[i] for i in data.test_idx[1].tolist()]

In [ ]:
# sub-table construction
df = pd.DataFrame({'Side_Effect_Index': data.test_et, 
                  'Side_Effect_Unique_ID': side_effect_idx,
                  'Side_Effect_Name': side_effect_name,
                  'Drug1_Index': data.test_idx[0],
                  'Drug1_Unique_ID': drug1_cid,
                  'Drug2_Index': data.test_idx[1], 
                  'Drug2_Unique_ID': drug2_cid,
                  'Prob_DF': score1,
                  'Prob_DDM_DF': score2,
                  'Prob_DDM_NN': score3,
                  'Prob_PPM_GGM_NN': score4,
                  'Prob_PPM_GGM_DDM_DF_Cat': score5,
                  'Prob_PPM_GGM_DDM_DF_Sum': score6
                })
df

,Side_Effect_Index,Side_Effect_Unique_ID,Side_Effect_Name,Drug1_Index,Drug1_Unique_ID,Drug2_Index,Drug2_Unique_ID,Prob_DF,Prob_DDM_DF,Prob_DDM_NN,Prob_PPM_GGM_NN,Prob_PPM_GGM_DDM_DF_Cat,Prob_PPM_GGM_DDM_DF_Sum
0,959,8924,cleft lip,0,2173,1,3345,0.500000,4.654672e-01,0.523101,5.000000e-01,4.735912e-01,5.403015e-01
1,959,8924,cleft lip,0,2173,2,5206,0.500000,3.349866e-02,0.075150,5.000000e-01,3.993516e-02,5.440357e-02
2,959,8924,cleft lip,0,2173,3,9433,0.043054,3.334496e-03,0.060729,5.000000e-01,3.314328e-03,6.755470e-03
3,959,8924,cleft lip,0,2173,4,3929,0.506104,5.733842e-02,0.189698,5.000000e-01,6.859240e-02,9.965137e-02
4,959,8924,cleft lip,0,2173,5,150610,0.500000,2.421660e-03,0.035472,5.000000e-01,8.205515e-04,2.305365e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28661215,1096,9952,febrile convulsion,641,9571074,643,2182,0.106131,7.552808e-11,0.000027,1.586276e-03,2.192506e-08,6.095405e-11
28661216,1096,9952,febrile convulsion,641,9571074,644,5482,0.053796,7.156607e-24,0.000027,5.000000e-01,1.866621e-08,5.165802e-13
28661217,1096,9952,febrile convulsion,642,4011,643,2182,0.069542,6.588929e-12,0.000006,3.954704e-09,1.060029e-08,1.278976e-10
28661218,1096,9952,febrile convulsion,642,4011,644,5482,0.286108,2.617764e-26,0.000006,2.489113e-06,2.096570e-08,5.809854e-12


## Writing results to CSV files in 2 chunks based on the iteration 

In [ ]:
if iteration == 1: # use this for the first iteration - save directly to csv file
  df.to_csv('analysis/evaluation_table_1.csv', index=False)

else if iteration <= 6: #use this otherwise - append to existing csv 
  df.to_csv('analysis/evaluation_table_1.csv', mode='a', header=False, index=False)

else if iteration==7:
  df.to_csv('analysis/evaluation_table_2.csv', index=False)

else:
  df.to_csv('analysis/evaluation_table_2.csv', mode='a', header=False, index=False)

## Combining the CSV chunk files 

In [ ]:
import pandas as pd

In [ ]:
df2 = pd.read_csv('analysis/evaluation_table_2.csv')

In [ ]:
df2.describe()

,Side_Effect_Index,Side_Effect_Unique_ID,Drug1_Index,Drug1_Unique_ID,Drug2_Index,Drug2_Unique_ID,Prob_DF,Prob_DDM_DF,Prob_DDM_NN,Prob_PPM_GGM_NN,Prob_PPM_GGM_DDM_DF_Cat,Prob_PPM_GGM_DDM_DF_Sum
count,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07,5.711475e+07
mean,9.590000e+02,1.176083e+05,2.143333e+02,3.320752e+05,4.296667e+02,3.306823e+05,3.086848e-01,2.006216e-01,2.368004e-01,4.242710e-01,1.963766e-01,2.003365e-01
std,7.938514e+01,2.267778e+05,1.519097e+02,1.236875e+06,1.519097e+02,1.271347e+06,2.620838e-01,2.729678e-01,2.783737e-01,2.439187e-01,2.717136e-01,2.772972e-01
min,8.220000e+02,7.680000e+02,0.000000e+00,8.500000e+01,1.000000e+00,8.500000e+01,2.363076e-12,0.000000e+00,8.496277e-10,0.000000e+00,2.228922e-36,2.215905e-32
25%,8.900000e+02,1.928400e+04,8.600000e+01,3.222000e+03,3.220000e+02,3.008000e+03,5.825280e-02,3.004166e-03,1.824277e-02,2.475053e-01,3.303863e-03,2.585272e-03
50%,9.590000e+02,3.258000e+04,1.880000e+02,4.506000e+03,4.560000e+02,4.440000e+03,2.601454e-01,5.108992e-02,1.033927e-01,5.000000e-01,4.716669e-02,4.548503e-02
75%,1.028000e+03,1.514630e+05,3.220000e+02,9.433000e+03,5.580000e+02,2.799100e+04,5.000000e-01,3.226005e-01,3.914230e-01,5.000000e-01,3.090489e-01,3.213737e-01
max,1.096000e+03,1.527411e+06,6.430000e+02,9.571074e+06,6.440000e+02,9.571074e+06,9.995364e-01,9.874603e-01,9.978272e-01,9.995290e-01,9.895920e-01,9.955140e-01


In [ ]:
df2.to_csv('analysis/evaluation_table_1.csv', mode='a', index=False, header=False)